In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df = pd.read_csv('iot23_test_binary.csv')

In [3]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = pd.get_dummies(df['label']).values

In [4]:
scaler = MinMaxScaler()
scaler.fit(X)
normalized_x = scaler.transform(X)
scaler.fit(Y)
normalized_y = scaler.transform(Y)

In [5]:
X = torch.from_numpy(normalized_x).float()
Y = torch.from_numpy(normalized_y).float()

In [6]:
class ResNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet, self).__init__()
        self.fc1 = nn.Linear(24, 96)
        self.bn1 = nn.BatchNorm1d(96)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(96, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.fc3 = nn.Linear(24, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out

In [7]:
class DenseNet(nn.Module):
    def __init__(self, num_classes=2):
        super(DenseNet, self).__init__()

        self.fc1 = nn.Linear(24, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.25)

        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.25)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.25)

        self.fc4 = nn.Linear(256, num_classes)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)

        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout2(out)

        out = self.fc3(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.dropout3(out)

        out = self.fc4(out)
        out = self.softmax(out)

        return out

In [8]:
# class Model1(nn.Module):
#     def __init__(self, input_shape, num_classes):
#         super(Model1, self).__init__()
#         self.fc1 = nn.Linear(input_shape, 64)
#         self.fc2 = nn.Linear(64, num_classes)
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         return x

class Model2(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# class Model3(nn.Module):
#     def __init__(self, input_shape, num_classes):
#         super(Model3, self).__init__()
#         self.fc1 = nn.Linear(input_shape, 32)
#         self.fc2 = nn.Linear(32, num_classes)
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         return x

class EnsembleModel(nn.Module):
    def __init__(self, input_shape, num_classes, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        self.fc1 = nn.Linear(len(models) * num_classes, num_classes)
        
    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))
        x = torch.cat(outputs, dim=1)
        x = self.fc1(x)
        x = torch.softmax(x, dim=1)
        return x

In [9]:
model = torch.load('stacking_binary.pt')

In [10]:

# Make predictions on the test set
y_pred = model(X)
y_pred = torch.argmax(y_pred, dim=1)

# Compute accuracy
_, normal_y_test = torch.max(Y.data,1)
accuracy = (y_pred == normal_y_test).float().mean()
print("Accuracy on test set:", accuracy.item())


Accuracy on test set: 0.5462999939918518
